# Evaluate Agents

In [ ]:
import os
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from stable_baselines3 import DQN, DDPG

from alphaQ.utils import download_ticker_data, train_test_split, plot_episodes, sharpe, save_to_s3, load_from_s3
from alphaQ.env import PortfolioEnv
from alphaQ.agent.features import FeatureExtractor
from alphaQ.agent.callbacks import EvalCallback
from alphaQ.agent.utils import AgentStrategy, load_model, display_attributes
from alphaQ.eval import evalu8, evaluate_baselines

import config
from config import TICKERS, MODELS, MODEL_PARAMS

warnings.filterwarnings('ignore')

rcParams = {
    'xtick.labelsize': 12,
    'ytick.labelsize': 12,
    'figure.figsize': (18, 9),
    'legend.fontsize': 13,
    'axes.labelsize': 14
}
plt.rcParams.update(rcParams)

In [ ]:
# Load ticker and market data, split into train, validation and test sets
esg_key = 'tradeGPT/prices/esg.pkl'
market_key = 'tradeGPT/prices/market.pkl'  # SPY

tickers = ["MSFT", "NVDA", "GOOGL", "LLY"]  # MSCI USA ESG Leaders Index (USD)

if False:
    data = download_ticker_data(tickers, start=config.START, end=config.END, columns=['Open', 'High', 'Low', 'Close']).dropna()
    save_to_s3(data, esg_key)
    market = download_ticker_data('SPY', start=config.START, end=config.END, columns=['Adj Close']).loc[data.index]
    save_to_s3(market, market_key)
else:
    data = load_from_s3(esg_key)
    market = load_from_s3(market_key)

train, val, test = train_test_split(data, train_years=12)
market_train, market_val, market_test = train_test_split(market, train_years=12)

In [ ]:
agents = []
model = 'dqn'  # 'ddpg'  

In [ ]:
agent_strategies = []
agents.append((model, MODELS[model].load("models/esg_dqn_low_lr/best_model.zip")))

# for all loaded agents
for model, agent in agents:
    
    # determing action space type
    action_space_type = config.ACTION_SPACE[model]

    # set up test environment
    test_env = PortfolioEnv(
        tickers=TICKERS,
        prices=test, 
        market_prices=market_test,
        window_length=config.WINDOW_LENGTH,
        trading_cost=config.COMMISSION_RATE,
        action_space_type=action_space_type,
#         render_mode='test'
    )
    
    print(model)
    # evaluate agent on test environment and plot results
    evalu8(agent, test_env)
    
    # display agent attributes
    display_attributes(agent, test_env.observation_space)    

    # wrap agent weights in universal portfolios interface for comparison
    strategy = AgentStrategy(test_env.record.actions, name=model.upper())
    agent_strategies.append(strategy)

In [ ]:
results, metrics = evaluate_baselines(test, custom_strategies=agent_strategies, market_data={'DJIA': market_test}, attributes=None)
metrics